# Cookbook: Mind Map

In this cookbook, we will demonstrate how to interact with files and create structured visual outputs like mind maps using an agent.

# Install Requirements

We will use [Qwen-Agent](https://github.com/QwenLM/Qwen-Agent) in this book. For demonstration, we will need at least the `mcp` functionality of Qwen-Agent.

In [ ]:
!pip3 install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
# `pip install -U qwen-agent` will install the minimal requirements.
# The optional requirements, specified in double brackets, are:
#   [gui] for Gradio-based GUI support;
#   [rag] for RAG support;
#   [code_interpreter] for Code Interpreter support;
#   [mcp] for MCP support.

In [ ]:
!pip3 install -U uv
# We will use mcp servers that run with the `uvx` command, so `uv` is required.

In [ ]:
!npm --version
# We also use mcp servers that run with the `npx` command, so node.js should be installed.

# Create a Document

Here, we create a sample document and save it as `example_document.md` for later use.

In [1]:
document = '''
# Document

## Introduction
- Brief overview of the document's purpose and main topics.

## Background
- Historical context and previous research.
- Key theories and models discussed.

## Methodology
- Detailed description of the methods used in the study.
- Explanation of data collection and analysis techniques.

## Results
- Presentation of the findings from the research.
- Statistical analysis and graphical representations.

## Discussion
- Interpretation of the results.
- Comparison with previous studies.
- Limitations and future research directions.

## Conclusion
- Summary of the main points covered in the document.
- Final thoughts and implications of the research.
'''.strip()

with open('example_document.md', 'w', encoding='utf-8') as f:
    f.write(document)

# Step 1: Create an Agent

Create an agent that is capable of 
- listing, reading and writing files (through the MCP server `@modelcontextprotocol/server-filesystem`).
- making mind maps (through the MCP server `mindmap-mcp-server`, source: [Mindmap MCP Server](https://github.com/YuChenSSR/mindmap-mcp-server)).

In [2]:
from qwen_agent.agents import Assistant
from qwen_agent.utils.output_beautify import typewriter_print
# `typewriter_print` prints streaming messages in a non-overlapping manner.

In [3]:
import dotenv
# Load environment variables from a .env file if it exists
dotenv.load_dotenv()
DASHSCOPE_API_KEY = dotenv.get_key(dotenv.find_dotenv(), 'DASHSCOPE_API_KEY')

In [4]:
llm_cfg = {
    'model': 'qwen3-32b',
    'model_server': 'dashscope',
    'api_key': DASHSCOPE_API_KEY # **fill your api key here**

    # Use a model service compatible with the OpenAI API, such as vLLM or Ollama:
    # 'model': 'Qwen3-8B',
    # 'model_server': 'http://localhost:8000/v1',  # base_url, also known as api_base
    # 'api_key': 'EMPTY'
}

tools = [{
    "mcpServers": {
        "filesystem": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                '.',
            ]
        },
        "mindmap": {
            "command": "uvx",
            "args": ["mindmap-mcp-server", "--return-type", "filePath"]
        }
    }
}]
agent = Assistant(
    llm=llm_cfg,
    function_list=tools
)

2025-06-04 10:56:11,558 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['filesystem', 'mindmap']
2025-06-04 10:56:11,567 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: filesystem
2025-06-04 10:56:12,344 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: mindmap


# Step 2: Number the Headings

We will add numbers to the Markdown headings to demonstrate how we can manipulate Markdown documents with the agent.

In [5]:
messages = []
messages += [{"role": "user", "content": "Read the example_document.md file in the current folder. In your reply, show me the content with the second-level headings numbered. (example: ## heading -> ## 1. heading)"}]

In [6]:
response_plain_text = ''
for ret_messages in agent.run(messages):
    # `ret_messages` will contain all subsequent messages, consisting of interleaved assistant messages and tool responses
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, let's see. The user wants me to read the example_document.md file in the current folder and show the content with second-level headings numbered. First, I need to figure out how to read the file. The available tools include filesystem-read_file, which seems appropriate here since it's a single file.

So, I'll use the filesystem-read_file function with the path set to "example_document.md". Once I get the content, I need to process it to number the second-level headings. The user provided an example where "## heading" becomes "## 1. heading". That suggests I need to find all lines starting with "##" and replace them with a numbered version.

Wait, but how to handle multiple headings? I need to keep track of the count. Each time I encounter a "##", increment a counter and prepend the number. But since I'm just showing the content with the modifications, maybe I can do this in the response after reading the file. However, the tools don't have a function for editing files, bu

In [7]:
messages += ret_messages

# Step 3: Create a Mind Map

We will create a mind map by prompting the agent, and it will call the corresponding tool.

In [8]:
messages += [{'role': 'user', 'content': 'Create a mind map with your previous markdown output. Show me the saved path.'}]

In [9]:
response_plain_text = ''
for ret_messages in agent.run(messages):
    # `ret_messages` will contain all subsequent messages, consisting of interleaved assistant messages and tool responses
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user wants me to create a mind map from the previous markdown output and show the saved path. Let me recall the tools available.

First, I need to take the modified markdown content with numbered headings. The user had the markdown content where each ## heading was converted to ## X. heading. The assistant provided that modified content.

Now, the user wants a mind map of that. The available tool is mindmap-convert_markdown_to_mindmap. This function takes markdown_content as input. So I can use that function here.

Wait, but do I need to pass the original markdown or the modified one with numbers? The user's previous response shows the modified version. Since the user wants the mind map based on the modified content, I should use that modified markdown.

But how do I get that content into the function? The assistant's last message included the modified markdown. I can take that content and pass it into the mindmap function.

Once the function is called, it should gene

In [ ]:
# We can preview the mind map in the browser!

# Use the GUI

We have explored the capabilities of the Qwen-Agent framework and Qwen models in processing files. We can also achieve this by using the GUI.

In [9]:
from qwen_agent.gui import WebUI

agent = Assistant(
    name="Qwen File Processing Assistant",
    description="I can help with your file processing needs, ask me anything!",
    llm=llm_cfg,
    function_list=tools
)

WebUI(agent).run()

2025-05-01 21:25:44,068 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['filesystem', 'mindmap']
2025-05-01 21:25:44,074 - mcp_manager.py - 245 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: filesystem
2025-05-01 21:25:45,110 - mcp_manager.py - 245 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: mindmap


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
